<a href="https://colab.research.google.com/github/RERobbins/data_science_266_sandbox/blob/main/4_QA_RAG_Self_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering Using Retrieval Augmented Generation With a Self-Querying Retriever

As we have worked with the vector database, it has become clear that it would be useful if we could take a look at a query and see if there was an obvious way to apply metadata filters when we retrieve information from it.  For example, if we ask a question about Company X and the database has information about Company X and Company Y with suitable metadata tags, we ought to be able to take a query that focuses on Company X and only look at information in the vector database that relates to Company X.

Once can imagine a variety of ways to approach this problem.  LangChain provides one mechanism, they call it a self-querying retriever, which they describe as follows:

> A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying VectorStore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

See [here](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/) for more detail.

This notebook picks up where we left off in the Retrieval Augmented Generation notebook.

The comments made about the use of generative models, embedding models and vector databases apply here.  Moreover, the code to either build your vector database or use one that you have kept is the same.

# Setup

## Environment Related Helpers

This portion of the notebook includes `install_if_needed` which will install a single package or list of packages with `pip` only if necessary, and `running_in_colab` a predicate that returns `True` if the notebook is running in Google Colab.

In [1]:
import importlib
import warnings
import textwrap

warnings.filterwarnings("ignore", category=UserWarning)


def install_if_needed(package_names):
    """
    Install one or more Python packages using pip if they are not already installed.

    Args:
        package_names (str or list): The name(s) of the package(s) to install.

    Returns:
        None
    """
    if isinstance(package_names, str):
        package_names = [package_names]

    for package_name in package_names:
        try:
            importlib.import_module(package_name)
            print(f"{package_name} is already installed.")
        except ImportError:
            !pip install --quiet {package_name}
            print(f"{package_name} has been installed.")


def running_in_colab():
    """
    Check if the Jupyter Notebook is running in Google Colab.

    Returns:
        bool: True if running in Google Colab, False otherwise.
    """
    try:
        import google.colab

        return True
    except ImportError:
        return False

## Mount Google Drive

By default, the data you create in Google Colaboratory does not persist from session to session.  Each session runs in a virtual machine and when that machine goes away, so does your data.  If you want your data to persist, you must store it outside the virtual machine. Google Drive can be used for that purpose.  We use it later in this notebook to store the OpenAI and Cohere API keys.

In [2]:
if running_in_colab():
    from google.colab import drive

    drive.mount("drive")

Mounted at drive


## API Keys

In [3]:
install_if_needed("python-dotenv")

python-dotenv has been installed.


In [4]:
import os
import getpass

from dotenv import load_dotenv, find_dotenv

def env_file_path(
    colab_path="/content/drive/MyDrive/.env", other_path=f"{find_dotenv()}"
):
    """
    Returns the appropriate file path for the environment variables file (.env) based on the execution environment.

    This function is designed to determine the correct path for the environment variables file
    depending on whether the code is running in Google Colab or in a different environment.

    Args:
        colab_path (str, optional): The file path for the environment variables file in Google Colab.
            Default is '/content/drive/MyDrive/.env'.

        other_path (str, optional): The file path for the environment variables file in other environments.
            Default is '/workspace/.env'.

    Returns:
        str: The file path for the environment variables file (.env).
    """

    return colab_path if running_in_colab() else other_path

In [5]:
load_dotenv(env_file_path())
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
COHERE_API_KEY = os.environ["COHERE_API_KEY"]

## Langchain

In [6]:
### Thie current version of LangChain when this notebook was
### last revised WAS 0.0.311.  That version introduced a bug
### the breaks the self-querying retriever code.
### To avoid that problem we use version 0.0.310 instead.

#install_if_needed("langchain")
!pip install --quiet langchain==0.0.310

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


## GPU Support (Optional)

In [7]:
import tensorflow as tf

print("GPU Available:", tf.config.list_physical_devices("GPU"))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:
install_if_needed("torch")
import torch

print("CUDA Available:", torch.cuda.is_available())

torch is already installed.
CUDA Available: True


## Embeddings

Instantiate the embeddings model.  

In [9]:
packages = [
    "openai",
    "cohere",
    "tiktoken",
    "transformers",
    "sentence_transformers",
]

install_if_needed(packages)

import openai, tiktoken

import cohere
from langchain.chat_models import ChatOpenAI
from langchain.llms import Cohere

from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.1 MB/s eta 0:00:00
openai has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 18.7 MB/s eta 0:00:00
cohere has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.0 MB/s eta 0:00:00
tiktoken has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
transformers has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.0 MB/s eta 0:00:00
sentence_transformers has been installed.


In [10]:
st_model_name = "multi-qa-mpnet-base-cos-v1"
st_embeddings_model = HuggingFaceEmbeddings(model_name=st_model_name)
st_tokenizer = AutoTokenizer.from_pretrained(f"sentence-transformers/{st_model_name}")

embeddings_model = st_embeddings_model

## Vector Database

In [11]:
install_if_needed("qdrant-client")
from langchain.vectorstores import Qdrant

import qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.4 which is incompatible.
qdrant-client has been installed.


# Generative Model

OpenAI trial accounts expire after three months and provide access to `gpt-3.5-turbo` but not `gpt-4`.  Paid OpenAI accounts permit use of `gpt-4` as well and do not expire.  Cohere trial accounts do not expire, but the API rate limiting is more significant than OpenAI trial account rate limiting.

Set the `LLM` variable below to reflect the generative model you want to use.  

The results in most of the examples below will vary with your choice.

In [12]:
# llm = Cohere(model="command", temperature=0)
LLM = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
# llm = ChatOpenAI(model="gpt-4", temperature=0)

# Load Documents, Split Into Chunks, Create Vector Database

**If you saved your Qdrant database when you worked on the vector database notebook you can skip this section and use the Load Persistent Vector Database section below.**

Otherwise, we use the following cells to download the privacy policies we have been working with and split them into chunks to be stored in the vector database.

In [13]:
install_if_needed(["pypdf", "unstructured"])

from langchain.document_loaders import PyPDFLoader, UnstructuredURLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.5/276.5 kB 5.1 MB/s eta 0:00:00
pypdf has been installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
unstructured has been installed.


In [14]:
import pandas as pd

policy_data = [
    ("Apple",
     "Privacy Policy",
     "https://www.apple.com/legal/privacy/pdfs/apple-privacy-policy-en-ww.pdf",
    ),
    ("Cohere", "Privacy Policy", "https://cohere.com/privacy"),
    ("Google",
     "Privacy Policy",
     "https://static.googleusercontent.com/media/www.google.com/en//intl/en/policies/privacy/google_privacy_policy_en.pdf",
    ),
    ("Hugging Face", "Privacy Policy", "https://huggingface.co/privacy"),
    ("Meta",
     "Privacy Policy",
     "https://about.fb.com/wp-content/uploads/2022/07/Privacy-Within-Metas-Integrity-Systems.pdf",
    ),
    ("Threads", "Privacy Policy", "https://terms.threads.com/privacy-policy"),
    ("TikTok",
     "Privacy Policy",
     "https://www.tiktok.com/legal/page/us/privacy-policy/en",
    ),]

columns = ["organization", "title", "url"]

policy_df = pd.DataFrame(policy_data, columns=columns)

In [15]:
def get_chunks(url, organization, title, chunk_size=2000, chunk_overlap=500):
    """
    This function takes a url to an organization's web page, organization name,
    and document title and returns chunks constructed from the target url.
    The function adds the url, the organization name and the document title
    as metadata to the chunks.

    Parameters:
    url (string): Target page.
    organization (string): Organization name.
    title: Document title.
    chunk_size (int, optional): Chunk size, default is 2000 characters.
    chunk_overlap (int, optional): Chunk overlap, default is 500 characters.

    Returns:
    list of chunks
    """

    # Use PyPDFLoader for pdf targets, otherwise UnstructuredURLLoader
    if os.path.splitext(url)[1] == ".pdf":
        loader = PyPDFLoader(url)
    else:
      loader = UnstructuredURLLoader([url])

    documents = loader.load()
    for document in documents:
        metadata = document.metadata
        metadata["url"] = url
        metadata["organization"] = organization
        metadata["title"] = title
        if metadata.get("page", None) is not None:
            metadata["page"] += 1

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    return text_splitter.split_documents(documents)


def explore_documents(documents):
    block_indent = "   "
    metadata = documents[0].metadata
    content = documents[0].page_content[:300] + ". . ."
    print(f"{metadata['organization']} {metadata['title']} {len(documents)} chunks")
    print("Truncated First chunk:")
    print(
        textwrap.fill(
            content,
            initial_indent=block_indent,
            subsequent_indent=block_indent,
            replace_whitespace=True,
        )
    )
    print()

In [16]:
chunks = []

for row in policy_df.itertuples(index=False):
    policy_chunks = get_chunks(row.url, row.organization, row.title)
    explore_documents(policy_chunks)
    chunks += policy_chunks

Apple Privacy Policy 18 chunks
Truncated First chunk:
   Apple Privacy Policy Apple’s Privacy Policy describes how Apple
   collects, uses, and shares your personal data. Updated December 22,
   2022  In addition to this Privacy Policy, we provide data and
   privacy information embedded in our products and certain features
   that ask to use your personal data. This. . .



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Cohere Privacy Policy 10 chunks
Truncated First chunk:
   Products  For Business  Developers  Research  Company  Try now
   Cohere Privacy Policy  Last Update: Aug 4, 2023  Cohere Inc.
   (“Cohere”) values and respects your privacy. We have prepared this
   privacy policy to explain the manner in which we collect, use, and
   disclose personal information through our. . .

Google Privacy Policy 20 chunks
Truncated First chunk:
   Privacy Policy Last modified: December 18, 2017 ( view archived
   versions ) (The hyperlinked examples are available at the end of
   this document.) There are many different ways you can use our
   services – to search for and share information, to communicate with
   other people or to create new content. Wh. . .

Hugging Face Privacy Policy 12 chunks
Truncated First chunk:
   Terms of Service  Privacy Policy  Content Policy  Code of Conduct
   Hugging Face Privacy Policy  🗓 Effective Date: March 28, 2023  We
   have implemented this Privacy Policy because 

In [17]:
print (f"There are {len(chunks)} chunks.")

There are 140 chunks.


In [18]:
%%time

collection_name = "my_collection"

vectordb = Qdrant.from_documents(
    documents = chunks,
    embedding = embeddings_model,
    location = ":memory:",
    collection_name = collection_name
    )

CPU times: user 3.98 s, sys: 751 ms, total: 4.73 s
Wall time: 9.74 s


In [19]:
# Confirm that we have the same number of vectors in the vector database as we have chunks.

assert vectordb.client.get_collection(collection_name).vectors_count == len(chunks)

# Load Persistent Vector Database

**If you saved your Qdrant database when you worked on the vector database notebook you can use this section instead of the Load Documents, Split Into Chunks, Create Vector Database section above.**

If you execute this code block more than once in a session you are likely to get an error indicating that your vector database is already accessed by another instance of Qdrant client and that if you require concurrent access, you should use Qdrant server instead.

The prior section should always work in lieu of this section.

In [ ]:
collection_name = "my_collection"
qdrant_database_location = "/content/drive/MyDrive/my_qdrant"

client = qdrant_client.QdrantClient(path=qdrant_database_location)

vectordb = Qdrant(client=client,
                   collection_name=collection_name,
                   embeddings=embeddings_model,)

In [ ]:
assert vectordb.client.get_collection(collection_name).vectors_count == 140

# Self-querying retriever

A self-querying retriever is one that, as the name suggests, has the ability to query itself. Specifically, given any natural language query, the retriever uses a query-constructing LLM chain to write a structured query and then applies that structured query to it's underlying vectorstore. This allows the retriever to not only use the user-input query for semantic similarity comparison with the contents of stored documented, but to also extract filters from the user query on the metadata of stored documents and to execute those filters.

The self-querying retriever's arguments include descriptions of the metadata fields and the document content.

In [20]:
install_if_needed("lark")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 3.5 MB/s eta 0:00:00
lark has been installed.


In [21]:
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

metadata_field_info = [
    AttributeInfo(
        name="organization",
        description="The company or organization that created the document.  It describes that company's policy.",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="title",
        description="The title of the document",
        type="string",
    ),
    AttributeInfo(
        name="url",
        description="The url for the document",
        type="string",
    ),
]
document_content_description = "A policy"

retriever = SelfQueryRetriever.from_llm(
    LLM,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True,
    enable_limit=True,
)

Now we build chain using our new retriever with the default RetrievalQA prompt and add a function to pretty-print results.

In [22]:
chain = RetrievalQA.from_chain_type(
    llm = LLM,
    retriever = retriever,
    return_source_documents = True,
)

def pretty_print_result(result):
  indent = "    "
  organizations = [document.metadata["organization"] for document in result['source_documents']]
  print()
  print(f"Query: {result['query']}")
  print(f"Organizations: {organizations}")
  print(f"Answer:")
  indent = "    "
  print (textwrap.fill(result['result'], initial_indent=indent, subsequent_indent=indent))


We examine both the metadata filter generated and the organization for the relevant documents retrieved.

As you look at the examples below and substitute your own you may discover that the approach is not consistently reliable.  In some cases, the system seems to fail to understand that a word in a query is an organization.  Sometimes revising the prompt a little bit to make that distinction more apparent helps.

Does this suggest that using our generative models to do entity extraction is perhaps not the best way to proceed?

In the next example, the system does not identify Meta as an organization.  Nevertheless, it is interesting to note that three of the four examples relate to Meta.  However, one does not.

In [23]:
result = chain("How does Meta protect my data?")
pretty_print_result(result)


Query: How does Meta protect my data?
Organizations: ['Meta', 'Cohere', 'Meta', 'Meta']
Answer:
    Meta implements reasonable administrative, technical, and physical
    measures to safeguard personal information against theft, loss,
    unauthorized access, use, modification, and disclosure. Access to
    personal information is restricted to employees and authorized
    service providers on a need-to-know basis. They only retain
    personal information for as long as it is operationally or legally
    necessary, and after that, they either destroy or anonymize the
    information. If you request access or updates to your personal
    information, Meta will direct you to the relevant customer. You
    may have the right to access, update, correct, delete, transfer,
    and object to the processing of your personal information. Meta is
    committed to privacy and conducts privacy reviews for new products
    and tools that involve user data. They evaluate privacy risks and
    make

However, when we change the query to make more explicit that we are talking about the company named Meta, the filter we want is generated and the documents are limited to Meta.

In [24]:
result = chain("How does the company named Meta protect my data?")
pretty_print_result(result)


Query: How does the company named Meta protect my data?
Organizations: ['Meta', 'Meta', 'Meta', 'Meta']
Answer:
    Meta protects your data through its commitment to privacy and its
    Privacy Review process. The company analyzes privacy alongside
    safety, security, and integrity concerns. New products and
    internal tools go through a privacy review, where experts evaluate
    privacy risks and make necessary changes before launch. This
    review is especially important for safety, security, and integrity
    tools that may use a range of data to detect and prevent harm.
    Meta follows eight privacy principles when discussing the
    appropriate use of personal data. These principles include purpose
    limitation, data minimization, and data retention. The company
    aims to process data only for a limited, clearly stated purpose
    that provides value to users. They collect and create the minimum
    amount of data required to support these purposes and retain data
    o

One of the subtle and even more interesting things about LangChain's self query retriever is that we can use it to allow the query to specify the number of documents to fetch.  We did that by passing `enable_limit=True` to the constructor.  See the relevant documentation [here](https://python.langchain.com/docs/modules/data_connection/retrievers/self_query/qdrant_self_query).

In the next example, the prompt asks for five examples, the query has `limit=5` and we get five results instead of the default of four.  Of course, saying we want five examples does not necessarily imply that we should see five source documents.  A single source document could contain multiple examples.

In [25]:
result = chain("How does the company named Meta protect my data?  I want five examples.")
pretty_print_result(result)


Query: How does the company named Meta protect my data?  I want five examples.
Organizations: ['Meta', 'Meta', 'Meta', 'Meta', 'Meta']
Answer:
    Meta protects user data through various measures. Here are five
    examples:  1. Privacy Review: Meta has a Privacy Review process in
    place for all new products and internal tools that handle user
    data. This review involves experts from legal, policy, and product
    teams who evaluate privacy risks associated with the project and
    make necessary changes to control those risks before launch.  2.
    Data Minimization: Meta follows the principle of data
    minimization, which means they collect and create only the minimum
    amount of data required to support the stated purposes. This
    ensures that unnecessary or excessive data is not collected or
    stored.  3. Data Retention: Meta retains user data only for as
    long as it is necessary to support the stated purposes. This helps
    in minimizing the storage of user data

Now, let's ask about two companies.  The filter seems to be doing the right thing. Notice the response regarding Cohere though.

In [26]:
result = chain("Do Apple or Cohere use cookies?")
pretty_print_result(result)


Query: Do Apple or Cohere use cookies?
Organizations: ['Apple', 'Cohere', 'Apple', 'Apple']
Answer:
    Yes, both Apple and Cohere use cookies. Apple uses cookies on
    their website to understand website activity, monitor and improve
    the website, and provide a customized experience. Cohere, on the
    other hand, may also use cookies on their website, but the
    provided context does not specifically mention Cohere's use of
    cookies.


When we write our prompt to make clear that Cohere is the name of a company and don't reference other companies, we get back sufficient detail to answer the question.  Depending on the way we word the prompt, the system sometimes fails to recognize that Cohere is the name of a company.  When referencing more than one company, perhaps we should be forcing the retriever to retrieve more documents.

In [27]:
result = chain("Does the company named Cohere use cookies?")
pretty_print_result(result)


Query: Does the company named Cohere use cookies?
Organizations: ['Cohere', 'Cohere', 'Cohere', 'Cohere']
Answer:
    Yes, the company named Cohere uses cookies on its website. The
    website's privacy policy states that they use cookies, pixel tags,
    and other tracking technologies to collect information about
    visitors' behavior on the website. These cookies are used to
    recognize the user's computer or device when they return to the
    website and to optimize the user experience.


Unlike the last example, which worked well, see what happens with a more natural phrasing of the question.  Not only does the system fail to recognize that Cohere is a company, it generates a filter on the word "cookies" as if that is an organization.  Finally, when there are no source documents returned, the systems responds in the affirmative.  But we don't know if that is a hallucination or not.

In [28]:
result = chain("Does Cohere use cookies?")
pretty_print_result(result)


Query: Does Cohere use cookies?
Organizations: []
Answer:
    Yes, Cohere uses cookies.


But what happens when we ask about a company not in our database?  The filter looks good, but the mechanism is blind to the absence of Microsoft data in the vector database.  Our relevant documents don't include Microsoft.  What if we asked about a pair of companies and the most relevant documents still came from only one?

In [29]:
result = chain ("Does Apple use cookies?  Does Microsoft use cookies?")
pretty_print_result(result)


Query: Does Apple use cookies?  Does Microsoft use cookies?
Organizations: ['Apple', 'Apple', 'Apple', 'Apple']
Answer:
    Yes, Apple uses cookies. The privacy policy states that Apple's
    websites, online services, interactive applications, and
    advertisements may use cookies and other technologies to better
    understand user behavior, facilitate and measure the effectiveness
    of advertisements and web searches, and enable network traffic to
    and from Apple's systems.  As for Microsoft, the given context
    does not provide any information about whether Microsoft uses
    cookies or not.


Up to this point we have not included prompts that looked to exclude information.  Let's try some below.  

In the first example, we get responses related to Cohere, Apple, Meta and Hugging Face.

In [30]:
result = chain("How do companies protect my data?")
pretty_print_result(result)


Query: How do companies protect my data?
Organizations: ['Cohere', 'Apple', 'Meta', 'Hugging Face']
Answer:
    Companies protect your data by implementing various safeguards and
    security measures. These measures may include administrative,
    technical, and physical controls to prevent unauthorized access,
    use, modification, and disclosure of your personal information.
    Access to your data is restricted to employees and authorized
    service providers who have a legitimate need to access it for
    their job responsibilities. Companies also follow data retention
    practices, keeping your personal information only for as long as
    necessary and either destroying or anonymizing it afterwards.
    Additionally, companies may have procedures in place to handle
    data subject requests, such as providing access to or updating
    personal information. It is important to note that while companies
    strive to use commercially acceptable means to protect your data,
    no

Lets revise the prompt to ask ask for five examples, cite the relevant companies, and limit itself to the supplied context.  The results are very interesting and impressive.

In [31]:
query = """
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.
"""

result = chain(query)
pretty_print_result(result)


Query: 
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.

Organizations: ['Cohere', 'Apple', 'Meta', 'Hugging Face', 'Apple']
Answer:
    Based on the provided context, here are five examples of how
    companies protect data:  1. Implementing Safeguards: Companies
    like Apple have implemented reasonable administrative, technical,
    and physical measures to safeguard personal information against
    theft, loss, and unauthorized access. (Source: Apple)  2.
    Restricting Access: Companies limit access to personal information
    on a need-to-know basis, ensuring that only employees and
    authorized service providers who require access for their job
    responsibilities can access the data. (Source: Apple)  3. Data
    Minimization: Companies collect and create the minimum amount of
    data requi

Now we revise the prompt to exclude Apple.  The filter looks good, it excludes Apple.

In [32]:
query = """
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.
I am not interested in information about Apple's practices.
"""

result = chain(query)
pretty_print_result(result)


Query: 
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.
I am not interested in information about Apple's practices.

Organizations: ['Cohere', 'Meta', 'Hugging Face', 'Google', 'TikTok']
Answer:
    1. Companies implement reasonable administrative, technical, and
    physical measures to safeguard personal information against theft,
    loss, and unauthorized access, use, modification, and disclosure.
    For example, Hugging Face, a company mentioned in the context,
    follows generally accepted industry standards and uses appropriate
    safeguards to protect personal information.  2. Companies restrict
    access to personal information on a need-to-know basis to
    employees and authorized service providers who require access to
    fulfill their job requirements. This ensures that only authorized

Finally, we revise the prompt to exclude both Hugging Face and Apple.

In [33]:
query = """
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.
I am not interested in information about Hugging Face or Apple.
"""

result = chain(query)
pretty_print_result(result)


Query: 
How do companies protect my data?
Please provide five examples.
When you give an example, please cite one or more companies that provide the protection you describe.
Do not describe any practice not included in the supplied context.
I am not interested in information about Hugging Face or Apple.

Organizations: ['Cohere', 'Meta', 'Google', 'TikTok', 'Meta']
Answer:
    Based on the provided context, here are five examples of how
    companies protect data:  1. Safeguards: Companies implement
    administrative, technical, and physical measures to safeguard
    personal information against theft, loss, and unauthorized access.
    For example, Google implements safeguards to protect user data.
    2. Access Control: Companies restrict access to personal
    information on a need-to-know basis, only allowing employees and
    authorized service providers who require access to fulfill their
    job requirements. For example, companies like Google and Hugging
    Face have access 

Let's try one more time to see how the system responds when we ask only about a company not represented in our collection.

In [34]:
query = """
How does Microsoft protect my data?
Limit your response to information in the supplied context.
"""

result = chain(query)
pretty_print_result(result)


Query: 
How does Microsoft protect my data?
Limit your response to information in the supplied context.

Organizations: []
Answer:
    Microsoft protects your data through various security measures,
    such as encryption, access controls, and regular security updates.
    They also have strict privacy policies in place to ensure that
    your data is not accessed or used without your consent.
    Additionally, Microsoft offers features like multi-factor
    authentication and data backup options to further enhance the
    protection of your data.


If you experiment with the self query mechanism you will likely conclude that it is fragile.  The implementations seem to vary by vector database as well.  For example, when this notebook was created, Chroma, a popular vector database for simple LangChain examples, does not support self query operations that result in the use of the `NOT` operator even though the query parser recognizes when that operator should be used and Chroma supports that operator directly.